In [1]:
import os
import nibabel as nb
import numpy as np
from scipy.sparse import lil_matrix

DS_atlases = ['DS00071', 'DS00195', 'DS00583', 'DS03231',
           'DS00096', 'DS00278', 'DS00833', 'DS06481', 'DS00108', 'DS00350',
           'DS01216', 'DS16784', 'DS00140', 'DS00446', 'DS01876', 'DS72784']
anat_atlases = ['AAL', 'slab1068', 'HarvardOxford', 'Talairach', 'desikan', 'JHU', 'CPAC200', 'slab907']

basepath = '/home/eric/atlases/labels'
outbase = '../../data/DS-anat/'

In [ ]:
for ds in DS_atlases:
    for anat in anat_atlases:
        ds_dat = nb.load(os.path.join(basepath, ds + '.nii.gz')).get_data()
        anat_dat = nb.load(os.path.join(basepath, anat + '.nii.gz')).get_data()
        dsregs = np.unique(ds_dat)
        dsregs = dsregs[dsregs > 0]
        anatregs = np.unique(anat_dat)
        
        overlapdat = lil_matrix((dsregs.shape[0], anatregs.shape[0]), dtype=np.float32)
        for didx, dsreg in enumerate(dsregs):
            dseq = (ds_dat == dsreg)
            N = dseq.sum()
            poss_regs = np.unique(anat_dat[dseq])
            for aidx, anatreg in enumerate(anatregs):
                if (anatreg in poss_regs):
                    # percent overlap is dseq and'd with the anatomical region voxelspace, summed and normalized
                    pover = np.logical_and(dseq, anat_dat == anatreg).sum()/float(N)
                    overlapdat[didx, aidx] = pover
                
        outf = os.path.join(outbase, "{}_{}.csv".format(ds, anat))
        with open(outf, 'w')  as f:
            anatstr = ["%d" % x for x in anatregs]
            f.write("dsreg," + ",".join(anatstr) + "\n")
            for idx, dsreg in enumerate(dsregs):
                datstr = ["%.4f" % x for x in overlapdat[idx,].toarray()[0,]]
                f.write(str(dsreg) + "," + ",".join(datstr) + "\n")
            f.close()